# Analyze the Survey Data

Analyze the survey data in `survey.csv` using the power of PYTHON!!

Also STATISTICS!!

And maybe some other made up stuff

## Installing

In a clean `python3` environment, run:

In [ ]:
!pip install -r requirements.txt